In [2]:
import sys
sys.path.append('/Users/mojgan/code/EPITECH/ZOIDBERG2.0/')

In [3]:
import torch
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision import datasets
from src.config import PATH_PERSO_TRAIN_NEW, PATH_PERSO_VALID_NEW, PATH_PERSO_TEST_NEW,PATH_PERSO_OUTPUTS
import torch.nn as nn
import torch.nn.functional as F
print("PyTorch version:", torch.__version__)

PyTorch version: 2.3.1


In [4]:
train_img_dir = PATH_PERSO_TRAIN_NEW
valid_img_dir = PATH_PERSO_VALID_NEW
test_img_dir = PATH_PERSO_TEST_NEW
path_out = PATH_PERSO_OUTPUTS

In [5]:
transform = transforms.Compose([
    transforms.Resize((150, 150)),
    transforms.ToTensor(),
])

In [6]:
validation_dataset = datasets.ImageFolder(valid_img_dir, transform=transform)
validation_loader = DataLoader(validation_dataset, batch_size=1, shuffle=True)

In [7]:
test_dataset = datasets.ImageFolder(test_img_dir, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=True)

In [9]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(32 * 75 * 75, 128)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(128, 3)  # 3 output classes

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = x.view(-1, 32 * 75 * 75)  # Flatten
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        x = F.softmax(x, dim=1)  # Corrected softmax dimension to 1
        return x


model_training = SimpleCNN()

In [10]:
state_dict = torch.load('/Users/mojgan/code/EPITECH/chest_Xray/outputs/models/2024-07-16_11-05-44/final_model.pth')

In [11]:
remapped_state_dict = {}
for key in state_dict:
    if key.startswith('0'):
        new_key = key.replace('0', 'conv1')
    elif key.startswith('4'):
        new_key = key.replace('4', 'fc1')
    elif key.startswith('7'):
        new_key = key.replace('7', 'fc2')
    else:
        new_key = key
    
    remapped_state_dict[new_key] = state_dict[key]

# Now load the model with remapped state_dict
model_training.load_state_dict(remapped_state_dict)

<All keys matched successfully>

In [12]:
# Convert integer labels to one-hot encoded vectors


def one_hot_encode(labels, num_classes):
    return F.one_hot(labels, num_classes).float()

In [13]:
model_training.eval()
criterion = nn.CrossEntropyLoss()  # For multi-class classification
val_loss = 0.0
correct = 0
total = 0

all_labels = []
all_outputs = []

with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model_training(inputs)
        loss = criterion(outputs, labels)
        val_loss += loss.item()
        _, res = torch.max(outputs, 1)
        correct += (res == labels).sum().item()
        total += labels.size(0)

        # Convert labels to one-hot encoding
        one_hot_labels = one_hot_encode(labels, num_classes=3)
        
        all_labels.append(one_hot_labels)
        all_outputs.append(outputs)

test_accuracy = correct / total
print(f"Validation Loss: {val_loss / len(test_loader)}, Validation Accuracy: {test_accuracy}")

Validation Loss: 0.9734918467708938, Validation Accuracy: 0.6503972758229285


negatif => sigmoid pour [0,1]

In [26]:
def confusion_matrix(y_true, y_pred, num_classes):
    cm = torch.zeros(num_classes, num_classes, dtype=torch.int64)
    for t, p in zip(y_true, y_pred):
        cm[t, p] += 1
    return cm


# Number of samples
num_classes = 3

# Generate random one-hot encoded true labels
true_labels_test = torch.argmax(torch.cat(all_labels), dim=1)

# Generate random predicted classes
predicted_classes = torch.argmax(torch.cat(all_outputs), dim=1)

# Calculate confusion matrix
cm = confusion_matrix(true_labels_test, predicted_classes, num_classes)
print(cm)

tensor([[144,  79,  15],
        [  0, 415,   3],
        [  2, 209,  14]])


In [33]:
# Calculate True Positives (TP), False Positives (FP), False Negatives (FN)
TP = cm.diag()
FP = cm.sum(dim=1) - TP
FN = cm.sum(dim=0) - TP
TN = cm.sum() - (FP + FN + TP)

# Calcul des totaux
TP_total = TP.sum().item()
FP_total = FP.sum().item()
FN_total = FN.sum().item()
TN_total = TN.sum().item()

# Calculate Precision, Recall, and F1-score
precision = TP.float() / (TP + FP).float()
recall = TP.float() / (TP + FN).float()
f1 = 2 * (precision * recall) / (precision + recall)

# Affichage des résultats
print(f"F1-score: {f1}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")

In [32]:
print("TP :", TP)
print("FP :", FP)
print("FN :", FN)
print("TN :", TN)
print("TP_total :", TP_total)
print("FP_total :", FP_total)
print("FN_total :", FN_total)
print("TN_total :", TN_total)

Matrice de confusion :
tensor([[144,  79,  15],
        [  0, 415,   3],
        [  2, 209,  14]])
TP : tensor([144, 415,  14])
FP : tensor([ 94,   3, 211])
FN : tensor([  2, 288,  18])
TN : tensor([641, 175, 638])
TP_total : 573
FP_total : 308
FN_total : 308
TN_total : 1454
